<a href="https://colab.research.google.com/github/kaniska/covid-19-hackathon/blob/analyze-streaming-data/StreamingSocialNetworkMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime
import json
import re
import os
import tweepy
from pprint import pprint as pp
import json
import os
import time

In [11]:
symptom_queries = ['shortness of breath' , 'sore throat' , 'headache' , 'respiratory virus' , 'bronchoalveolar lavage fluid' , 'ground glass opacities' , 'Bluish lips or face', 'Real-Time RT-PCR Diagnostic Panel', 'myalgia', 'fever 100', 'fever 38C', 'rhinorrhea', 'dyspnea', 'Nausea']
mental_anxiety_keywords = ['fear', 'worry', 'mental health', 'insomnia', 'depression', 'uncertainty', \
                  'sick', 'anxiety', 'loneliness', 'sad', 'stress']

In [12]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [13]:
def checkStates(input):
	print("_7 "+input)
	if input:
		print("_7")
		abbrevAndNames = json.dumps(states)
		print("_7a")
		regex = re.compile(r'\b(' + '|'.join(abbrevAndNames) + r')\b')
		print("_7 b")
		location = regex.search(input)
		print("_8 ")
		if location:
			print("_9")
			return states.get(location.group(0), location.group(0))


In [14]:
import json
import re
##abbrevAndNames = str(states.keys()) + str(states.values())
#print(abbrevAndNames)
#regex = re.compile(r'\b(' + '|'.join(abbrevAndNames) + r')\b')
#location = regex.search('Manila, Philippines')
#if location:
#    print( states.get(location.group(0), location.group(0)))

In [20]:
def convertTweetToJson4(tweet):
    dict4stream = {}
    print("_2")
    location = ""
    dict4stream["tweetId"] = tweet["id"]
    dict4stream["tweetText"] = tweet["text"]
    dict4stream["timestamp"] = tweet["created_at"]
    place = tweet['place']
    print("_3")
    resultTweet = ""
    if place:
        location = place["full_name"]
        print("_4")
    else:
        user = tweet['user']
        if user:
            location = user["location"]
            print("_5")
    if location:
        dict4stream["location"] = location
    print("_6")    
    resultTweet =json.dumps(dict4stream)
    print("_10 "+resultTweet)
    return resultTweet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:

class TweetStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        try:
            json4 = convertTweetToJson4(status._json)
            if json4:
                timestr = time.strftime("%Y%m%d") + "-streamedCovidJson"
                filePath = "drive/My Drive/TweetStreams/"+timestr
                with open(filePath, 'a+') as f:
                    f.write(json4)
                    f.write("\n")
                    return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            return True
    

    def on_error(self, status_code):
        print(status_code)
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False

    
    def on_error(self, status_code):
        print(status_code)
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False

In [17]:
consumer_key = ''
consumer_secret = ''  
access_token = ''
access_token_secret = ''
auth1 = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth1.set_access_token(access_token, access_token_secret)

In [ ]:
tweetsStream = tweepy.Stream(auth = auth1, listener=TweetStreamListener())
tweetsStream.filter(track= mental_anxiety_keywords)